<a href="https://colab.research.google.com/github/analyticsariel/projects/blob/master/How_to_Get_Total_Population_from_Census_API_using%C2%A0Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to Get Total Population from Census API using Python

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Ariel Herrera arielherrera@analyticsariel.com |
| External References   | API |
| Input Datasets        | Source name |
| Output Datasets       | Source name |
| Input Data Source     | Pandas DataFrame |
| Output Data Source    | Pandas DataFrame |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 1st Sep 2022 | Ariel Herrera | Create notebook. |

## Getting Started
1. Copy this notebook -> File -> Save a Copy in Drive
2. Directions

## Useful Resources
- [American Community Survey 5-Year Data API](https://www.census.gov/data/developers/data-sets/acs-5year.html)
- [Request Census API Key](https://api.census.gov/data/key_signup.html)
- [Google Colab Cheat Sheet](https://towardsdatascience.com/cheat-sheet-for-google-colab-63853778c093)

## <font color="blue">Install Packages</font>

## <font color="blue">Imports</font>

In [1]:
from google.colab import output, drive, files # specific to Google Colab
import pandas as pd
import numpy as np
import plotly.express as px
import requests
import warnings

# settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

## <font color="blue">Functions</font>

## <font color="blue">Locals & Constants</font>

In [2]:
############
# OPTIONAL #
############

# mount drive
drive.mount('/content/drive', force_remount=False)

# data location
file_dir = '/content/drive/My Drive/Colab Data/input/' # optional

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# read in api key file
df_api_keys = pd.read_csv(file_dir + 'api_keys.csv')

# get keys
census_api_key = df_api_keys.loc[df_api_keys['API'] =='census']['KEY'].iloc[0] # replace this with your own key
rapid_api_key = df_api_keys.loc[df_api_keys['API'] =='rapid']['KEY'].iloc[0] # replace this with your own key

## <font color="blue">Data</font>

### <font color="green">Section #1 - API Requests</font> 💻
This section will cover how to make API requests to the [American Community Survey 5-Year Data API](https://www.census.gov/data/developers/data-sets/acs-5year.html) It demonstrates how to modify your search based on different parameters.

In [4]:
# dictionary of all variables
variable = 'B01001_001E'
year = '2020'

#### <font color="purple">1. National Region</font> 👨‍👩‍👧‍👦

In [5]:
# get data by nation
url = 'https://api.census.gov/data/{0}/acs/acs5?get=NAME,{1}&for=us:*&key={2}'\
  .format(year, variable, census_api_key)
# request data
response = requests.request("GET", url)
# view status code
response.status_code

200

In [6]:
# view data
response.text

'[["NAME","B01001_001E","us"],\n["United States","326569308","1"]]'

In [7]:
# transform to JSON object
response.json() 

[['NAME', 'B01001_001E', 'us'], ['United States', '326569308', '1']]

In [8]:
# read population data as a dataframe
df_national = pd.DataFrame(response.json()[1:], columns=response.json()[0])
df_national

,NAME,B01001_001E,us
0,United States,326569308,1


In [9]:
# rename columns
df_national = df_national.rename(columns={'B01001_001E': 'total_population'})
df_national

,NAME,total_population,us
0,United States,326569308,1


#### <font color="purple">2. State Region</font> 👨‍👩‍👧‍👦

In [10]:
# get data by state
url = 'https://api.census.gov/data/{0}/acs/acs5?get=NAME,{1}&for=state:*&key={2}'\
  .format(year, variable, census_api_key)
response = requests.request("GET", url)
response.status_code

200

In [11]:
# read population data
df_state = pd.DataFrame(response.json()[1:], columns=response.json()[0])\
  .rename(columns={'B01001_001E': 'total_population'})
print('Num of rows:', len(df_state))
df_state.head()

Num of rows: 52


,NAME,total_population,state
0,Pennsylvania,12794885,42
1,California,39346023,06
2,West Virginia,1807426,54
3,Utah,3151239,49
4,New York,19514849,36


#### <font color="purple">3. Metro Region</font> 👨‍👩‍👧‍👦

In [12]:
# get data by metro
url = 'https://api.census.gov/data/{0}/acs/acs5?get=NAME,{1}&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&key={2}'\
  .format(year, variable, census_api_key)
response = requests.request("GET", url)
response.status_code

200

In [13]:
# read population data
df_metro = pd.DataFrame(response.json()[1:], columns=response.json()[0])\
  .rename(columns={'B01001_001E': 'total_population'})
print('Num of rows:', len(df_metro))
df_metro.head()

Num of rows: 939


,NAME,total_population,metropolitan statistical area/micropolitan statistical area
0,"Aberdeen, SD Micro Area",42864,10100
1,"Aberdeen, WA Micro Area",73769,10140
2,"Abilene, TX Metro Area",171354,10180
3,"Ada, OK Micro Area",38385,10220
4,"Adrian, MI Micro Area",98310,10300


#### <font color="purple">4. County Region</font> 👨‍👩‍👧‍👦

In [14]:
# get data by county
url = 'https://api.census.gov/data/{0}/acs/acs5?get=NAME,{1}&for=county:*&in=state:*&key={2}'\
  .format(year, variable, census_api_key)
response = requests.request("GET", url)
response.status_code

200

In [15]:
# read population data
df_county = pd.DataFrame(response.json()[1:], columns=response.json()[0])\
  .rename(columns={'B01001_001E': 'total_population'})
print('Num of rows:', len(df_county))
df_county.head()

Num of rows: 3221


,NAME,total_population,state,county
0,"Autauga County, Alabama",55639,01,001
1,"Baldwin County, Alabama",218289,01,003
2,"Barbour County, Alabama",25026,01,005
3,"Bibb County, Alabama",22374,01,007
4,"Blount County, Alabama",57755,01,009


### <font color="green">Section #2 - Loops</font> 🏙
Get data for multiple years for a given region

In [16]:
# list of years for Census API
year_list = ['2015', '2016', '2017', '2018', '2019', '2020']

#### <font color="purple">1. Census API</font>

In [17]:
# iterate through list of years
df_list = []
for year in year_list:
  # get total population by state
  url = 'https://api.census.gov/data/{0}/acs/acs5?get=NAME,{1}&for=state:*&key={2}'\
    .format(year, variable, census_api_key)
  response = requests.request("GET", url)
  _df = pd.DataFrame(response.json()[1:], columns=response.json()[0])
  _df['year'] = year
  df_list.append(_df)

# combine responses into a single dataframe
df_census = pd.concat(df_list)\
  .rename(columns={'B01001_001E': 'total_population'})
df_census['total_population'] = df_census['total_population'].astype(int)
print('Num of rows:', len(df_census))
df_census.head()

Num of rows: 312


,NAME,total_population,state,year
0,Mississippi,2988081,28,2015
1,Missouri,6045448,29,2015
2,Montana,1014699,30,2015
3,Nebraska,1869365,31,2015
4,Nevada,2798636,32,2015


In [18]:
# filter on state
state_name = 'Florida'
df_census_state = df_census.loc[df_census['NAME'] == state_name]
df_census_state

,NAME,total_population,state,year
37,Florida,19645772,12,2015
37,Florida,19934451,12,2016
36,Florida,20278447,12,2017
37,Florida,20598139,12,2018
9,Florida,20901636,12,2019
7,Florida,21216924,12,2020


#### <font color="purple">2. US Housing Market Data API</font> 🏘

### <font color="green">Section #1 - API Requests</font> 💻
Get enriched housing and economic datasets by signing up for the [US Housing Market Data API](https://bit.ly/3AHH7sY).

In [19]:
# get enriched dataset of housing and economic data
url = "https://us-housing-market-data.p.rapidapi.com/getStateEnriched"

querystring = {"state_code":"FL"}

headers = {
	"X-RapidAPI-Key": rapid_api_key,
	"X-RapidAPI-Host": "us-housing-market-data.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)
df_state_e = pd.DataFrame.from_dict(response.json(), orient='index')
print('Num of rows: {}'.format(len(df_state_e)))
print('Num of columns: {}'.format(len(df_state_e.columns)))
df_state_e.tail()

Num of rows: 127
Num of columns: 70


,fred.date,fred.total_population,fred.median_household_income,fred.median_list_price,fred.median_inventory,fred.median_days_on_market,fred.rental_vacancy_rate,fred.home_ownership_rate,fred.home_value_index,fred.unemployment_rate,fred.building_permits_housing_units,fred.gdp,redfin.period_begin,redfin.period_end,redfin.period_duration,redfin.region_type,redfin.region_type_id,redfin.table_id,redfin.is_seasonally_adjusted,redfin.region,redfin.city,redfin.state,redfin.state_code,redfin.property_type,redfin.property_type_id,redfin.median_sale_price,redfin.median_sale_price_mom,redfin.median_sale_price_yoy,redfin.median_list_price,redfin.median_list_price_mom,redfin.median_list_price_yoy,redfin.median_ppsf,redfin.median_ppsf_mom,redfin.median_ppsf_yoy,redfin.median_list_ppsf,redfin.median_list_ppsf_mom,redfin.median_list_ppsf_yoy,redfin.homes_sold,redfin.homes_sold_mom,redfin.homes_sold_yoy,redfin.pending_sales,redfin.pending_sales_mom,redfin.pending_sales_yoy,redfin.new_listings,redfin.new_listings_mom,redfin.new_listings_yoy,redfin.inventory,redfin.inventory_mom,redfin.inventory_yoy,redfin.months_of_supply,redfin.months_of_supply_mom,redfin.months_of_supply_yoy,redfin.median_dom,redfin.median_dom_mom,redfin.median_dom_yoy,redfin.avg_sale_to_list,redfin.avg_sale_to_list_mom,redfin.avg_sale_to_list_yoy,redfin.sold_above_list,redfin.sold_above_list_mom,redfin.sold_above_list_yoy,redfin.price_drops,redfin.price_drops_mom,redfin.price_drops_yoy,redfin.off_market_in_two_weeks,redfin.off_market_in_two_weeks_mom,redfin.off_market_in_two_weeks_yoy,redfin.parent_metro_region,redfin.parent_metro_region_metro_code,redfin.last_updated
122,2022-03-01,21781128.0,57435.0,449950.0,36544.0,37.0,6.5,67.2,366088,3.2,13267.0,1226297.5,2022-03-01,2022-03-31,30,state,4,19,f,Florida,None,Florida,FL,All Residential,-1,388500,0.040755,0.227207,421300.0,0.040109,0.246048,245.0,0.041102,0.278124,261.0,0.025556,0.273816,45866,0.304457,-0.108342,42547.0,0.184164,-0.069136,50564.0,0.183947,0.001684,57300.0,0.024239,-0.206635,1.2,-0.4,-0.2,24.0,-4.0,-15.0,1.001569,0.006494,0.020949,0.407208,0.055881,0.191613,0.145183,0.032356,0.004067,0.552688,-0.000608,0.071801,South Region,None,2022-08-14 14:44:22
123,2022-04-01,21781128.0,57435.0,472990.0,39429.0,35.0,6.5,67.2,376793,3.0,13511.0,1226297.5,2022-04-01,2022-04-30,30,state,4,19,f,Florida,None,Florida,FL,All Residential,-1,400000,0.029864,0.226987,434500.0,0.031373,0.256163,251.0,0.030187,0.275046,267.0,0.025489,0.276946,42746,-0.068126,-0.164366,39861.0,-0.063218,-0.057637,49186.0,-0.027445,-0.020336,61836.0,0.078880,-0.131212,1.4,0.2,0.0,22.0,-2.0,-10.0,1.005024,0.003457,0.018989,0.438684,0.031499,0.171374,0.162155,0.016957,0.036844,0.536246,-0.016431,0.031280,South Region,None,2022-08-14 14:44:22
124,2022-05-01,21781128.0,57435.0,490768.0,49266.0,32.0,6.5,67.2,387543,2.9,12635.0,1226297.5,2022-05-01,2022-05-31,30,state,4,19,f,Florida,None,Florida,FL,All Residential,-1,410100,0.025220,0.235568,437900.0,0.007925,0.246245,257.0,0.019121,0.278774,267.0,0.001300,0.252400,42895,0.003486,-0.086174,39409.0,-0.011339,-0.052805,52213.0,0.061542,0.071146,70165.0,0.134695,-0.004582,1.6,0.2,0.1,22.0,0.0,-8.0,1.004126,-0.000897,0.013423,0.433407,-0.005277,0.118196,0.220979,0.058824,0.095227,0.504711,-0.031535,-0.003551,South Region,None,2022-08-14 14:44:22
125,2022-06-01,21781128.0,57435.0,498500.0,60479.0,34.0,6.5,67.2,396607,2.8,12002.0,1226297.5,2022-06-01,2022-06-30,30,state,4,19,f,Florida,None,Florida,FL,All Residential,-1,410500,0.000970,0.203545,437800.0,-0.000265,0.245859,255.0,-0.007644,0.234818,266.0,-0.007345,0.237070,41279,-0.037673,-0.189065,36429.0,-0.075617,-0.138672,53916.0,0.032616,0.082281,82737.0,0.179178,0.173391,2.0,0.4,0.6,23.0,1.0,-5.0,0.998515,-0.005611,0.003686,0.390223,-0.043185,0.036530,0.296615,0.075635,0.148652,0.442105,-0.062606,-0.051901,South Region,None,2022-08-14 14:44:22
126,2022-07-01,21781128.0,57435.0,485000.0,76833.0,36.0,6.5,67.2,402837,2.7,9985.0,1226297.5,2022-07-01,2022-07-31,30,stat

### <font color="green">Section #3 - Visualization</font> 📈
Visualize housing and economic data trends over time

Source [US Census Bureau Estimate Total Population](https://api.census.gov/data/2020/acs/acs5/variables/B01001_001E.json)

In [20]:
fig = px.line(df_census_state, x='year', y='total_population', title='Population by Year for {}'.format(state_name))
fig.show()

Source [FRED US Census Bureau Resident Population (Not Seasonally Adjusted)](https://fred.stlouisfed.org/series/FLPOP)

In [21]:
df_plot = df_state_e.copy()
df_plot['month'] = df_plot.apply(lambda x: x['fred.date'].split('-')[1], axis=1) # get month from date
df_plot = df_plot.loc[df_plot['month'] == '01'] # select first month of year - january
fig = px.line(df_plot, x='fred.date', y='fred.total_population', title='Population by Year for {}'.format(state_name))
fig.show()

## <font color="blue">Output</font>

In [22]:
# # download file
# df.to_csv('output.csv', index=False)
# files.download('output.csv')

# End Notebook